In [22]:
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, ZeroPadding2D, MaxPooling2D, Conv3D
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.optimizers import Adam
from keras.constraints import unit_norm
import time
import json
import os
import warnings
import math

%matplotlib inline

warnings.filterwarnings(action='ignore')

In [3]:
def read_jsons(folder):
    jsons = [json for json in os.listdir(folder) if json.endswith(".json")]
    
    return jsons

In [4]:
def generateRandomPoint():
    return np.random.randint(1080, size = (25, 2))

In [5]:
def generateRandomLength():
    return np.random.randint(200, 500, size = 1)

In [6]:
def getNewLocations(index, distances, scaler, original, finalPoints, first):
    k = 0
    
    while k < 30:
        for j in range(len(distances)):
            if k == 30:
                break
                
            tmp = copy.deepcopy(original)
            
            for x in range(original.shape[0]):
                if index < 2:
                    tmp[x][index] = tmp[x][index] + (distances[j] * scaler)
                
                if index == 2:
                    tmp[x][0] = tmp[x][0] + distances[j]
                    tmp[x][1] = tmp[x][1] + distances[j]
                
                if index == 3:
                    tmp[x][0] = tmp[x][0] + distances[j]
                    tmp[x][1] = tmp[x][1] - distances[j]
                
                if index == 4:
                    tmp[x][0] = tmp[x][0] - distances[j]
                    tmp[x][1] = tmp[x][1] + distances[j]
                
                if index == 5:
                    tmp[x][0] = tmp[x][0] - distances[j]
                    tmp[x][1] = tmp[x][1] - distances[j]
                    
            if not first:
                finalPoints = np.concatenate((finalPoints, tmp), axis = 0)
            else:
                finalPoints = copy.deepcopy(tmp)
                first = False
                
            k += 1

    return finalPoints, first

In [7]:
def generateTrainingData():
    
    frequencies = np.linspace(3, 15, 13, dtype=int)
    
    k = 0
    l  = 0
    
    for j in range(5000):
        original = generateRandomPoint()
        distance = generateRandomLength()
        
        finalPoints = []
        y_points = []
        first = True
        
        l += 1
        
        for i in range(len(frequencies)):
            frequency = frequencies[i]

            div = int(math.floor(frequency / 2)) + frequency % 2
            
            first_half = np.linspace(0, distance, div, dtype = int)
            second_half = np.linspace(distance, 0, div, dtype = int)
            
            if frequency % 2 == 1:
                second_half = second_half[1:]
            else:
                first_half = first_half[:-1]

            distances = np.concatenate((first_half, second_half), axis = 0)
            

            finalPoints, first = getNewLocations(0, distances, 1, original, finalPoints, first)

            finalPoints, first = getNewLocations(0, distances, -1, original, finalPoints, first)

            finalPoints, first = getNewLocations(1, distances, 1, original, finalPoints, first)

            finalPoints, first = getNewLocations(1, distances, -1, original, finalPoints, first)
            
            finalPoints, first = getNewLocations(2, distances, -1, original, finalPoints, first)

            finalPoints, first = getNewLocations(3, distances, -1, original, finalPoints, first)

            finalPoints, first = getNewLocations(4, distances, -1, original, finalPoints, first)

            finalPoints, first = getNewLocations(5, distances, -1, original, finalPoints, first)

            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            y_points = np.append(y_points, frequency)
            
        with open('Repetition/RepetitionData' + str(j) + '.json', 'w') as fp:
            json.dump(dict(enumerate(finalPoints.tolist())), fp)
            
        with open('RepetitionResult/RepetitionResult' + str(j) + '.json', 'w') as fp:
            json.dump(dict(enumerate(y_points.tolist())), fp)
                
#     return finalPoints.reshape(13 * l * 8, 30, 50, 1), y_points    

In [8]:
def read_data(jsons, folder):
    data_points = []
    
    for i in range(len(jsons)):
        
        with open((folder + jsons[i])) as f:
            data = json.load(f)
            
        for k in data:
            data_points.append(data[k])
            
    return data_points

In [9]:
def loadGeneratedData(jsons, folder):
            
    data = read_data(jsons, folder)
    
    return data

In [10]:
def returnBatch(x_jsons, y_jsons):
    x_batch = x_jsons[:10]
    y_batch = y_jsons[:10]
    
    x_remain = x_jsons[10:]
    y_remain = y_jsons[10:]
    
    return x_batch, y_batch, x_remain, y_remain

In [11]:
def modelCreation():
    model = Sequential()
    model.add(Dense(64, activation = 'relu', input_shape = (30, 50, 1)))
#     model.add(MaxPooling2D((2,2)))
    
    model.add(Dense(100, activation = 'relu'))
#     model.add(MaxPooling2D((2,2)))
    
    model.add(Dense(150, activation = 'relu'))
#     model.add(MaxPooling2D((2,2)))
    
    model.add(Dense(200, activation = 'relu'))

#     model.add(Dense(80, activation = 'relu'))

#     model.add(Dense(90, activation = 'relu'))
    
#     model.add(Dense(90, activation = 'relu'))

    model.add(Flatten())
    
    model.add(Dense(200, activation = 'relu'))
#     model.add(Dropout(0.5))
    
    model.add(Dense(200, activation = 'relu'))
#     model.add(Dropout(0.5))
    
    model.add(Dense(16, activation='softmax'))
    
    return model

In [12]:
def trainModel(model, x_jsons, y_jsons):
    
    length = len(x_jsons)
    
    acc = []
    loss = []
    val_acc = []
    val_loss = []
    x_axis = []
    time_val = 0
    
    fig = plt.figure()
    ax = fig.add_subplot(221)
    ab = fig.add_subplot(222)
    ac = fig.add_subplot(223)
    ad = fig.add_subplot(224)
    ax.set_title('Acc')
    ab.set_title('Loss')
    ac.set_title('Val_acc')
    ad.set_title('Val_loss')
    plt.ion()
    
    fig.show()
    fig.canvas.draw()
    
    original_x_jsons = read_jsons('Repetition/')
    original_y_jsons = read_jsons('RepetitionResult/')
    
    for i in range(10):
        
        if i != 0:
            x_jsons = copy.deepcopy(original_x_jsons)
            y_jsons = original_y_jsons
        
        while x_jsons != []:
            x_data, y_data, x_jsons, y_jsons = returnBatch(x_jsons, y_jsons)

            x_raw = loadGeneratedData(x_data, 'Repetition/')
            y_raw = loadGeneratedData(y_data, 'RepetitionResult/')

            x_raw = StandardScaler().fit_transform(x_raw)
            
            X = np.array(x_raw)
            Y = np.array(y_raw)

            X = X.reshape(1040, 30, 50, 1)
            Y = Y.reshape(1040, 1)
            
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 111)

            y_train = to_categorical(y_train)
            y_test = to_categorical(y_test)

            history = model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=20, verbose=False)

            acc.append(history.history['acc'][-1])
            loss.append(history.history['loss'][-1])
            val_acc.append(history.history['val_acc'][-1])
            val_loss.append(history.history['val_loss'][-1])

            ax.clear()
            ab.clear()
            ac.clear()
            ad.clear()
            ax.set_title('Acc')
            ab.set_title('Loss')
            ac.set_title('Val_acc')
            ad.set_title('Val_loss')
            ax.plot(acc, '-o')
            ab.plot(loss,'-o')
            ac.plot(val_acc,'-o')
            ad.plot(val_loss,'-o')
            fig.canvas.draw()
            time.sleep(0.5)
            
            with open('Remaining/RemainingDeep.json', 'w') as fp:
                json.dump(dict(enumerate(x_jsons)), fp)
            
            with open('Remaining/RemainingResultDeep.json', 'w') as fp:
                json.dump(dict(enumerate(y_jsons)), fp)
                
            with open('Result/acc_shallow.json', 'w') as fp:
                json.dump(dict(enumerate(acc)), fp)
            
            with open('Result/loss_shallow.json', 'w') as fp:
                json.dump(dict(enumerate(loss)), fp)
                
            with open('Result/val_acc_shallow.json', 'w') as fp:
                json.dump(dict(enumerate(val_acc)), fp)
                
            with open('Result/val_loss_shallow.json', 'w') as fp:
                json.dump(dict(enumerate(val_loss)), fp)
            
            
            model.save('model/shallow_repetition.h5')

In [13]:
def loadRemainingData():
    with open('Remaining/Remaining.json', 'r') as fp:
        x_json = json.load(fp)

    with open('Remaining/RemainingResult.json', 'r') as fp:
        y_json = json.load(fp)
    
    
    X = []
    Y = []
    
    for keys in x_json:
        X.append(x_json[keys])
    
    for keys in y_json:
        Y.append(y_json[keys])
        
    return X, Y

In [14]:
def plotGraphs(fileNames):
    
    for i in range(len(fileNames)):
        with open('Result/' + fileNames[i] + '.json', 'r') as fp:
            data = json.load(fp)
            
        X = []
        
        for keys in data:
            X.append(data[keys])
            
        plt.plot(X)
        plt.show()

In [15]:
# generateTrainingData()

In [16]:
x_jsons = read_jsons('Repetition/')
y_jsons = read_jsons('RepetitionResult/')

In [17]:
# len(y_jsons)

In [18]:
model = modelCreation()
# model = load_model('model/deep_repetition.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
# x_jsons, y_jsons = loadRemainingData()

In [20]:
model.compile(optimizer = Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# trainModel(model, x_jsons, y_jsons)

In [23]:
fileNames = ['acc', 'loss', 'val_acc', 'val_loss']
plotGraphs(fileNames)

KeyboardInterrupt: 